## Scraping

In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
# Initialize the WebDriver with headless option
options = webdriver.ChromeOptions()
options.add_argument('--headless')
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

In [56]:
# Function to scrape data for a given year
def scrape_data_for_year(year):
    try:
        # Open the target URL
        url = f'https://www.spotrac.com/nfl/cash/{year}'
        browser.get(url)

        # Wait for the page to load completely
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.NAME, "year")))

        # Find the year dropdown and select the year
        select_year = Select(browser.find_element(By.NAME, "year"))
        select_year.select_by_visible_text(str(year))

        # Allow some time for the page to refresh after selecting a year
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='table dataTable']/tbody/tr")))

        # Scrape the data
        teams = []
        ranks = []
        records = []
        signed_players = []
        avg_ages = []
        active_cash = []
        dead_cash = []
        total_cash = []

        # Find all the rows in the table
        rows = browser.find_elements(By.XPATH, "//table[@class='table dataTable']/tbody/tr")

        # Loop through each row and extract the required data
        for row in rows:
            rank = row.find_element(By.XPATH, "./td[1]").text
            team = row.find_element(By.XPATH, "./td[2]").text
            record = row.find_element(By.XPATH, "./td[3]").text
            signed = row.find_element(By.XPATH, "./td[4]").text
            avg_age = row.find_element(By.XPATH, "./td[5]").text
            active = row.find_element(By.XPATH, "./td[6]").text
            dead = row.find_element(By.XPATH, "./td[7]").text
            total = row.find_element(By.XPATH, "./td[8]").text
            
            ranks.append(rank)
            teams.append(team)
            records.append(record)
            signed_players.append(signed)
            avg_ages.append(avg_age)
            active_cash.append(active)
            dead_cash.append(dead)
            total_cash.append(total)

        # Create a DataFrame from the extracted data
        data = {
            "Year": [year] * len(ranks),  # Add the year to each row for identification
            "Rank": ranks,
            "Team": teams,
            "Record": records,
            "Signed Players": signed_players,
            "Avg Age": avg_ages,
            "Active Cash": active_cash,
            "Dead Cash": dead_cash,
            "Total Cash": total_cash
        }

        df = pd.DataFrame(data)

        return df

    except Exception as e:
        print(f"An error occurred while scraping data for year {year}: {e}")
        return pd.DataFrame()

# List of years to scrape
years = [2020, 2021, 2022, 2023, 2024]

# Initialize an empty list to store all the DataFrames
all_data = []

# Loop through each year and scrape the data
for year in years:
    df_year = scrape_data_for_year(year)
    all_data.append(df_year)

# Concatenate all DataFrames into one
final_df = pd.concat(all_data, ignore_index=True)

# Print or save the final DataFrame
print(final_df)

# Optionally, you can save the DataFrame to a CSV file
final_df.to_csv("nfl_salary_cash_2020_2024.csv", index=False)

# Close the browser
browser.quit()

     Year Rank Team  Record Signed Players Avg Age   Active Cash    Dead Cash  \
0    2020    1  HOU  4-12-0             55    26.3  $192,489,580  $16,986,836   
1    2020    2   NO  12-4-0             53    27.0  $233,445,421   $9,000,166   
2    2020    3  DAL  6-10-0             52    25.9  $159,376,968  $21,631,582   
3    2020    4  LAC   7-9-0             55    25.5  $201,860,615   $6,506,526   
4    2020    5   SF  6-10-0             55    26.4  $139,826,064  $20,917,873   
..    ...  ...  ...     ...            ...     ...           ...          ...   
155  2024   28  LAC  11-6-0             53    26.6  $217,240,689   $6,675,070   
156  2024   29  SEA  10-7-0             52    26.1  $180,217,001  $22,243,030   
157  2024   30  PIT  10-7-0             51    27.6  $181,565,548   $6,560,159   
158  2024   31  LAR  10-7-0             53    25.9  $201,928,187   $7,621,992   
159  2024   32   LV  4-13-0             53    25.6  $116,277,872  $16,060,450   

       Total Cash  
0    $2

In [57]:
data = pd.read_csv('nfl_salary_cash_2020_2024.csv')
data

,Year,Rank,Team,Record,Signed Players,Avg Age,Active Cash,Dead Cash,Total Cash
0,2020,1,HOU,4-12-0,55,26.3,"$192,489,580","$16,986,836","$261,735,595"
1,2020,2,NO,12-4-0,53,27.0,"$233,445,421","$9,000,166","$253,584,935"
2,2020,3,DAL,6-10-0,52,25.9,"$159,376,968","$21,631,582","$249,805,299"
3,2020,4,LAC,7-9-0,55,25.5,"$201,860,615","$6,506,526","$247,147,613"
4,2020,5,SF,6-10-0,55,26.4,"$139,826,064","$20,917,873","$243,983,860"
...,...,...,...,...,...,...,...,...,...
155,2024,28,LAC,11-6-0,53,26.6,"$217,240,689","$6,675,070","$238,941,583"
156,2024,29,SEA,10-7-0,52,26.1,"$180,217,001","$22,243,030","$229,031,876"
157,2024,30,PIT,10-7-0,51,27.6,"$181,565,548","$6,560,159","$225,450,918"
158,2024,31,LAR,10-7-0,53,25.9,"$201,928,187","$7,621,992","$219,121,282"


## Data Integration

In [60]:
import pandas as pd

In [61]:
scraped_salary = pd.read_csv('nfl_salary_cash_2020_2024.csv')
scraped_salary

,Year,Rank,Team,Record,Signed Players,Avg Age,Active Cash,Dead Cash,Total Cash
0,2020,1,HOU,4-12-0,55,26.3,"$192,489,580","$16,986,836","$261,735,595"
1,2020,2,NO,12-4-0,53,27.0,"$233,445,421","$9,000,166","$253,584,935"
2,2020,3,DAL,6-10-0,52,25.9,"$159,376,968","$21,631,582","$249,805,299"
3,2020,4,LAC,7-9-0,55,25.5,"$201,860,615","$6,506,526","$247,147,613"
4,2020,5,SF,6-10-0,55,26.4,"$139,826,064","$20,917,873","$243,983,860"
...,...,...,...,...,...,...,...,...,...
155,2024,28,LAC,11-6-0,53,26.6,"$217,240,689","$6,675,070","$238,941,583"
156,2024,29,SEA,10-7-0,52,26.1,"$180,217,001","$22,243,030","$229,031,876"
157,2024,30,PIT,10-7-0,51,27.6,"$181,565,548","$6,560,159","$225,450,918"
158,2024,31,LAR,10-7-0,53,25.9,"$201,928,187","$7,621,992","$219,121,282"


In [62]:
team_data = pd.read_csv('nfl_team_data.csv')
team_data

,Date,DOW,WT,LT,WTS,LTS,Type,Season
0,9/10/20,Thu,Kansas City Chiefs,Houston Texans,34,20,Regular Season,2020
1,9/13/20,Sun,Seattle Seahawks,Atlanta Falcons,38,25,Regular Season,2020
2,9/13/20,Sun,Buffalo Bills,New York Jets,27,17,Regular Season,2020
3,9/13/20,Sun,Las Vegas Raiders,Carolina Panthers,34,30,Regular Season,2020
4,9/13/20,Sun,Chicago Bears,Detroit Lions,27,23,Regular Season,2020
...,...,...,...,...,...,...,...,...
1403,1/19/25,Sun,Philadelphia Eagles,Los Angeles Rams,28,22,Playoff,2024
1404,1/19/25,Sun,Buffalo Bills,Baltimore Ravens,27,25,Playoff,2024
1405,1/26/25,Sun,Philadelphia Eagles,Washington Commanders,55,23,Playoff,2024
1406,1/26/25,Sun,Kansas City Chiefs,Buffalo Bills,32,29,Playoff,2024


In [63]:
team_mapping = {
    "HOU": "Houston Texans",
    "NO": "New Orleans Saints",
    "DAL": "Dallas Cowboys",
    "LAC": "Los Angeles Chargers",
    "SF": "San Francisco 49ers",
    "PHI": "Philadelphia Eagles",
    "LV": "Las Vegas Raiders",
    "CIN": "Cincinnati Bengals",
    "MIA": "Miami Dolphins",
    "PIT": "Pittsburgh Steelers",
    "BAL": "Baltimore Ravens",
    "GB": "Green Bay Packers",
    "BUF": "Buffalo Bills",
    "TEN": "Tennessee Titans",
    "NYJ": "New York Jets",
    "SEA": "Seattle Seahawks",
    "KC": "Kansas City Chiefs",
    "LA": "Los Angeles Rams",
    "NE": "New England Patriots",
    "CHI": "Chicago Bears",
    "MIN": "Minnesota Vikings",
    "IND": "Indianapolis Colts",
    "ATL": "Atlanta Falcons",
    "CAR": "Carolina Panthers",
    "DET": "Detroit Lions",
    "JAC": "Jacksonville Jaguars",
    "NYG": "New York Giants",
    "WAS": "Washington Commanders",
    "ARI": "Arizona Cardinals",
    "CLE": "Cleveland Browns",
    "TB": "Tampa Bay Buccaneers"
}

In [64]:
scraped_salary['Team Name'] = scraped_salary['Team'].replace(team_mapping)
scraped_salary

,Year,Rank,Team,Record,Signed Players,Avg Age,Active Cash,Dead Cash,Total Cash,Team Name
0,2020,1,HOU,4-12-0,55,26.3,"$192,489,580","$16,986,836","$261,735,595",Houston Texans
1,2020,2,NO,12-4-0,53,27.0,"$233,445,421","$9,000,166","$253,584,935",New Orleans Saints
2,2020,3,DAL,6-10-0,52,25.9,"$159,376,968","$21,631,582","$249,805,299",Dallas Cowboys
3,2020,4,LAC,7-9-0,55,25.5,"$201,860,615","$6,506,526","$247,147,613",Los Angeles Chargers
4,2020,5,SF,6-10-0,55,26.4,"$139,826,064","$20,917,873","$243,983,860",San Francisco 49ers
...,...,...,...,...,...,...,...,...,...,...
155,2024,28,LAC,11-6-0,53,26.6,"$217,240,689","$6,675,070","$238,941,583",Los Angeles Chargers
156,2024,29,SEA,10-7-0,52,26.1,"$180,217,001","$22,243,030","$229,031,876",Seattle Seahawks
157,2024,30,PIT,10-7-0,51,27.6,"$181,565,548","$6,560,159","$225,450,918",Pittsburgh Steelers
158,2024,31,LAR,10-7-0,53,25.9,"$201,928,187","$7,621,992","$219,121,282",LAR


In [65]:
team_data.rename(columns={'WT':'Winning Team Name','LT':'Losing Team','Season':'Year','WTS':'Winning Team Score','LTS':'Losing Team Score','DOW':'Day of Week'},inplace=True)
team_data

,Date,Day of Week,Winning Team Name,Losing Team,Winning Team Score,Losing Team Score,Type,Year
0,9/10/20,Thu,Kansas City Chiefs,Houston Texans,34,20,Regular Season,2020
1,9/13/20,Sun,Seattle Seahawks,Atlanta Falcons,38,25,Regular Season,2020
2,9/13/20,Sun,Buffalo Bills,New York Jets,27,17,Regular Season,2020
3,9/13/20,Sun,Las Vegas Raiders,Carolina Panthers,34,30,Regular Season,2020
4,9/13/20,Sun,Chicago Bears,Detroit Lions,27,23,Regular Season,2020
...,...,...,...,...,...,...,...,...
1403,1/19/25,Sun,Philadelphia Eagles,Los Angeles Rams,28,22,Playoff,2024
1404,1/19/25,Sun,Buffalo Bills,Baltimore Ravens,27,25,Playoff,2024
1405,1/26/25,Sun,Philadelphia Eagles,Washington Commanders,55,23,Playoff,2024
1406,1/26/25,Sun,Kansas City Chiefs,Buffalo Bills,32,29,Playoff,2024


In [66]:
team_data['Team Name']=team_data['Winning Team Name']
team_data

,Date,Day of Week,Winning Team Name,Losing Team,Winning Team Score,Losing Team Score,Type,Year,Team Name
0,9/10/20,Thu,Kansas City Chiefs,Houston Texans,34,20,Regular Season,2020,Kansas City Chiefs
1,9/13/20,Sun,Seattle Seahawks,Atlanta Falcons,38,25,Regular Season,2020,Seattle Seahawks
2,9/13/20,Sun,Buffalo Bills,New York Jets,27,17,Regular Season,2020,Buffalo Bills
3,9/13/20,Sun,Las Vegas Raiders,Carolina Panthers,34,30,Regular Season,2020,Las Vegas Raiders
4,9/13/20,Sun,Chicago Bears,Detroit Lions,27,23,Regular Season,2020,Chicago Bears
...,...,...,...,...,...,...,...,...,...
1403,1/19/25,Sun,Philadelphia Eagles,Los Angeles Rams,28,22,Playoff,2024,Philadelphia Eagles
1404,1/19/25,Sun,Buffalo Bills,Baltimore Ravens,27,25,Playoff,2024,Buffalo Bills
1405,1/26/25,Sun,Philadelphia Eagles,Washington Commanders,55,23,Playoff,2024,Philadelphia Eagles
1406,1/26/25,Sun,Kansas City Chiefs,Buffalo Bills,32,29,Playoff,2024,Kansas City Chiefs


In [67]:
combined_nfl_data = pd.merge(team_data,scraped_salary,on=["Team Name","Year"])
combined_nfl_data

,Date,Day of Week,Winning Team Name,Losing Team,Winning Team Score,Losing Team Score,Type,Year,Team Name,Rank,Team,Record,Signed Players,Avg Age,Active Cash,Dead Cash,Total Cash
0,9/10/20,Thu,Kansas City Chiefs,Houston Texans,34,20,Regular Season,2020,Kansas City Chiefs,25,KC,14-2-0,51,26.1,"$182,213,498","$7,823,872","$202,794,195"
1,9/13/20,Sun,Seattle Seahawks,Atlanta Falcons,38,25,Regular Season,2020,Seattle Seahawks,18,SEA,12-4-0,52,26.6,"$190,854,794","$10,804,732","$221,174,179"
2,9/13/20,Sun,Buffalo Bills,New York Jets,27,17,Regular Season,2020,Buffalo Bills,15,BUF,13-3-0,52,26.7,"$206,918,396","$10,187,390","$223,199,805"
3,9/13/20,Sun,Las Vegas Raiders,Carolina Panthers,34,30,Regular Season,2020,Las Vegas Raiders,7,LV,8-8-0,53,26.3,"$193,781,662","$8,765,225","$234,936,034"
4,9/13/20,Sun,Chicago Bears,Detroit Lions,27,23,Regular Season,2020,Chicago Bears,12,CHI,8-8-0,53,26.9,"$193,338,682","$8,116,625","$228,516,276"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,1/19/25,Sun,Philadelphia Eagles,Los Angeles Rams,28,22,Playoff,2024,Philadelphia Eagles,3,PHI,14-3-0,53,25.7,"$294,433,688","$15,079,844","$328,556,211"
1275,1/19/25,Sun,Buffalo Bills,Baltimore Ravens,27,25,Playoff,2024,Buffalo Bills,14,BUF,13-4-0,53,26.9,"$262,492,400","$16,268,495","$285,254,387"
1276,1/26/25,Sun,Philadelphia Eagles,Washington Commanders,55,23,Playoff,2024,Philadelphia Eagles,3,PHI,14-3-0,53,25.7,"$294,433,688","$15,079,844","$328,556,211"
1277,1/26/25,Sun,Kansas City Chiefs,Buffalo Bills,32,29,Playoff,2024,Kansas City Chiefs,11,KC,15-2-0,53,26.5,"$266,722,742","$15,314,769","$293,739,868"


## Cleaning

In [69]:
combined_nfl_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
combined_nfl_data.fillna(method='bfill', inplace=True)  # Backward fill missing values

combined_nfl_data.drop_duplicates(inplace=True)

combined_nfl_data['Team Name'] = combined_nfl_data['Team Name'].str.strip()  # Remove leading/trailing spaces
combined_nfl_data['Year'] = combined_nfl_data['Year'].astype(int)  # Ensure 'Year' is an integer

print(combined_nfl_data)

combined_nfl_data.to_csv("cleaned_nfl_data.csv", index=False)

         Date Day of Week    Winning Team Name            Losing Team  \
0     9/10/20         Thu   Kansas City Chiefs         Houston Texans   
1     9/13/20         Sun     Seattle Seahawks        Atlanta Falcons   
2     9/13/20         Sun        Buffalo Bills          New York Jets   
3     9/13/20         Sun    Las Vegas Raiders      Carolina Panthers   
4     9/13/20         Sun        Chicago Bears          Detroit Lions   
...       ...         ...                  ...                    ...   
1274  1/19/25         Sun  Philadelphia Eagles       Los Angeles Rams   
1275  1/19/25         Sun        Buffalo Bills       Baltimore Ravens   
1276  1/26/25         Sun  Philadelphia Eagles  Washington Commanders   
1277  1/26/25         Sun   Kansas City Chiefs          Buffalo Bills   
1278   2/9/25         Sun  Philadelphia Eagles     Kansas City Chiefs   

      Winning Team Score  Losing Team Score            Type  Year  \
0                     34                 20  Regular S

/var/folders/zd/46rdsbcd33b6hgg4dy4k7d2m0000gn/T/ipykernel_79111/584488171.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_nfl_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
/var/folders/zd/46rdsbcd33b6hgg4dy4k7d2m0000gn/T/ipykernel_79111/584488171.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_nfl_data.fillna(method='bfill', inplace=True)  # Backward fill missing values


,Date,Day of Week,Winning Team Name,Losing Team,Winning Team Score,Losing Team Score,Type,Year,Team Name,Rank,Team,Record,Signed Players,Avg Age,Active Cash,Dead Cash,Total Cash
0,9/10/20,Thu,Kansas City Chiefs,Houston Texans,34,20,Regular Season,2020,Kansas City Chiefs,25,KC,14-2-0,51,26.1,"$182,213,498","$7,823,872","$202,794,195"
1,9/13/20,Sun,Seattle Seahawks,Atlanta Falcons,38,25,Regular Season,2020,Seattle Seahawks,18,SEA,12-4-0,52,26.6,"$190,854,794","$10,804,732","$221,174,179"
2,9/13/20,Sun,Buffalo Bills,New York Jets,27,17,Regular Season,2020,Buffalo Bills,15,BUF,13-3-0,52,26.7,"$206,918,396","$10,187,390","$223,199,805"
3,9/13/20,Sun,Las Vegas Raiders,Carolina Panthers,34,30,Regular Season,2020,Las Vegas Raiders,7,LV,8-8-0,53,26.3,"$193,781,662","$8,765,225","$234,936,034"
4,9/13/20,Sun,Chicago Bears,Detroit Lions,27,23,Regular Season,2020,Chicago Bears,12,CHI,8-8-0,53,26.9,"$193,338,682","$8,116,625","$228,516,276"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,1/19/25,Sun,Philadelphia Eagles,Los Angeles Rams,28,22,Playoff,2024,Philadelphia Eagles,3,PHI,14-3-0,53,25.7,"$294,433,688","$15,079,844","$328,556,211"
1275,1/19/25,Sun,Buffalo Bills,Baltimore Ravens,27,25,Playoff,2024,Buffalo Bills,14,BUF,13-4-0,53,26.9,"$262,492,400","$16,268,495","$285,254,387"
1276,1/26/25,Sun,Philadelphia Eagles,Washington Commanders,55,23,Playoff,2024,Philadelphia Eagles,3,PHI,14-3-0,53,25.7,"$294,433,688","$15,079,844","$328,556,211"
1277,1/26/25,Sun,Kansas City Chiefs,Buffalo Bills,32,29,Playoff,2024,Kansas City Chiefs,11,KC,15-2-0,53,26.5,"$266,722,742","$15,314,769","$293,739,868"
